In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
import joblib
import numpy as np

In [20]:
# Cargar datos
data = pd.read_csv('data_evaluacion.csv')


In [21]:
# Preprocesamiento
data_dummies = pd.get_dummies(data, drop_first=True)
target_column = '<=50K_>50K'
if target_column not in data_dummies.columns:
    raise KeyError(f"La columna '{target_column}' no se encuentra en los datos preprocesados.")

X = data_dummies.drop(target_column, axis=1)
y = data_dummies[target_column]

In [22]:
# Imputar valores NaN con la media de las columnas
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)


In [23]:
# Verificar si aún existen valores NaN
if np.isnan(X).any():
    raise ValueError("Todavía hay valores NaN en los datos después de la imputación.")


In [24]:
# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [25]:
# Escalar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [26]:
# Entrenar el modelo
naive_bayes = GaussianNB()
naive_bayes.fit(X_train_scaled, y_train)

GaussianNB()

In [27]:
# Función para evaluar el modelo
def evaluate_model(model, X_train, y_train, X_test, y_test):
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)

    train_accuracy = np.mean(train_pred == y_train)
    test_accuracy = np.mean(test_pred == y_test)

    print(f"Precisión en entrenamiento: {train_accuracy:.4f}")
    print(f"Precisión en prueba: {test_accuracy:.4f}")

    if train_accuracy - test_accuracy > 0.1:
        print("Posible overfitting")
    elif test_accuracy - train_accuracy > 0.1:
        print("Posible underfitting")
    else:
        print("Modelo bien ajustado")

In [28]:
# Evaluar el modelo
print("Naive Bayes")
print("Confusión en entrenamiento:")
print(confusion_matrix(y_train, naive_bayes.predict(X_train_scaled)))
print("Reporte de clasificación en entrenamiento:")
print(classification_report(y_train, naive_bayes.predict(X_train_scaled)))

print("Confusión en prueba:")
print(confusion_matrix(y_test, naive_bayes.predict(X_test_scaled)))
print("Reporte de clasificación en prueba:")
print(classification_report(y_test, naive_bayes.predict(X_test_scaled)))

Naive Bayes
Confusión en entrenamiento:
[[12718 13270]
 [  569  7631]]
Reporte de clasificación en entrenamiento:
              precision    recall  f1-score   support

       False       0.96      0.49      0.65     25988
        True       0.37      0.93      0.52      8200

    accuracy                           0.60     34188
   macro avg       0.66      0.71      0.59     34188
weighted avg       0.82      0.60      0.62     34188

Confusión en prueba:
[[5622 5544]
 [ 218 3269]]
Reporte de clasificación en prueba:
              precision    recall  f1-score   support

       False       0.96      0.50      0.66     11166
        True       0.37      0.94      0.53      3487

    accuracy                           0.61     14653
   macro avg       0.67      0.72      0.60     14653
weighted avg       0.82      0.61      0.63     14653



In [29]:
# Evaluar underfitting y overfitting
evaluate_model(naive_bayes, X_train_scaled, y_train, X_test_scaled, y_test)

Precisión en entrenamiento: 0.5952
Precisión en prueba: 0.6068
Modelo bien ajustado
